__Run basic contamination screening and fastqc. perform mapping and alignment, following this tutorial https://www.biostars.org/p/41951/__

In [1]:
%cd ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/


/local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen


In [4]:
#make a list of filenames to use for later loops
!ls *R1.fastq.gz > filenames
!sed -i'' -e 's/_R1.fastq.gz//g' filenames #remove the extension

In [ ]:
#screen for contamination and filter out tagged reads simultaneously, filter 1 means keep only reads that mapped exclusively to the reference for A. percula
!for i in /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/*.fastq.gz; do fastq_screen --tag --filter 1 --outdir /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/ $i; done

Using fastq_screen v0.13.0
Reading configuration from '/local/home/katrinac/miniconda3/envs/commandline/share/fastq-screen-0.13.0-0/fastq_screen.conf'
Aligner (--aligner) not specified, but BWA path and index files found: mapping with BWA
Using '/local/home/katrinac/miniconda3/envs/commandline/bin/bwa' as BWA path
Adding database Amphiprionpercula
Using 60 threads for searches
Option --subset set to 0: processing all reads in FASTQ files
Processing CAP10_APPC_34.05_R1.fastq.gz
Not making data subset
Searching CAP10_APPC_34.05_R1.fastq.gz_temp_subset.fastq against Amphiprionpercula
Filtering CAP10_APPC_34.05_R1.tagged.fastq.gz
Filtering files with filter '1'
Filtering CAP10_APPC_34.05_R1.tagged.fastq.gz
Processing complete

gzip: stdout: Broken pipe
Using fastq_screen v0.13.0
Reading configuration from '/local/home/katrinac/miniconda3/envs/commandline/share/fastq-screen-0.13.0-0/fastq_screen.conf'
Aligner (--aligner) not specified, but BWA path and index files found: mapping with BWA
Us

In [ ]:
#run fastqc 
!for i in /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/*tagged_filter.fastq.gz; do fastqc -t 32 *tagged_filter.fastq.gz  $i; done

Started analysis of CAP10_APPC_34.05_R1.tagged_filter.fastq.gz
Approx 5% complete for CAP10_APPC_34.05_R1.tagged_filter.fastq.gz
Started analysis of CAP10_APPC_34.05_R2.tagged_filter.fastq.gz
Approx 10% complete for CAP10_APPC_34.05_R1.tagged_filter.fastq.gz
Approx 5% complete for CAP10_APPC_34.05_R2.tagged_filter.fastq.gz
Approx 15% complete for CAP10_APPC_34.05_R1.tagged_filter.fastq.gz
Approx 10% complete for CAP10_APPC_34.05_R2.tagged_filter.fastq.gz
Started analysis of CAP10_APPC_34.06_R1.tagged_filter.fastq.gz
Approx 20% complete for CAP10_APPC_34.05_R1.tagged_filter.fastq.gz
Approx 15% complete for CAP10_APPC_34.05_R2.tagged_filter.fastq.gz
Approx 5% complete for CAP10_APPC_34.06_R1.tagged_filter.fastq.gz
Approx 25% complete for CAP10_APPC_34.05_R1.tagged_filter.fastq.gz
Approx 20% complete for CAP10_APPC_34.05_R2.tagged_filter.fastq.gz
Started analysis of CAP10_APPC_34.06_R2.tagged_filter.fastq.gz
Approx 30% complete for CAP10_APPC_34.05_R1.tagged_filter.fastq.gz
Approx 10% com

Approx 5% complete for CAP10_APPC_36.04_R1.tagged_filter.fastq.gz
Approx 10% complete for CAP10_APPC_35.11_R2.tagged_filter.fastq.gz
Approx 5% complete for CAP10_APPC_35.14_R1.tagged_filter.fastq.gz
Approx 40% complete for CAP10_APPC_35.07_R2.tagged_filter.fastq.gz
Analysis complete for CAP10_APPC_34.06_R1.tagged_filter.fastq.gz
Approx 90% complete for CAP10_APPC_34.06_R2.tagged_filter.fastq.gz
Approx 60% complete for CAP10_APPC_35.07_R1.tagged_filter.fastq.gz
Approx 20% complete for CAP10_APPC_35.11_R1.tagged_filter.fastq.gz
Approx 10% complete for CAP10_APPC_36.04_R1.tagged_filter.fastq.gz
Started analysis of CAP10_APPC_36.04_R2.tagged_filter.fastq.gz
Approx 65% complete for CAP10_APPC_35.07_R1.tagged_filter.fastq.gz
Started analysis of CAP10_APPC_36.06_R1.tagged_filter.fastq.gz
Approx 15% complete for CAP10_APPC_34.14_R1.tagged_filter.fastq.gz
Approx 45% complete for CAP10_APPC_35.07_R2.tagged_filter.fastq.gz
Approx 15% complete for CAP10_APPC_35.11_R2.tagged_filter.fastq.gz
Approx 

Approx 50% complete for CAP10_APPC_35.11_R2.tagged_filter.fastq.gz
Approx 65% complete for CAP10_APPC_35.11_R1.tagged_filter.fastq.gz
Approx 75% complete for CAP10_APPC_36.04_R1.tagged_filter.fastq.gz
Approx 60% complete for CAP10_APPC_36.06_R2.tagged_filter.fastq.gz
Approx 60% complete for CAP10_APPC_36.04_R2.tagged_filter.fastq.gz
Started analysis of CAP11_APPC_38.14_R2.tagged_filter.fastq.gz
Approx 35% complete for CAP10_APPC_36.06_R1.tagged_filter.fastq.gz
Approx 20% complete for CAP10_APPC_35.14_R2.tagged_filter.fastq.gz
Approx 10% complete for CAP11_APPC_38.12_R1.tagged_filter.fastq.gz
Approx 30% complete for CAP11_APPC_37.10_R1.tagged_filter.fastq.gz
Approx 25% complete for CAP10_APPC_35.14_R1.tagged_filter.fastq.gz
Approx 30% complete for CAP11_APPC_37.10_R2.tagged_filter.fastq.gz
Approx 80% complete for CAP10_APPC_36.04_R1.tagged_filter.fastq.gz
Approx 65% complete for CAP10_APPC_36.06_R2.tagged_filter.fastq.gz
Approx 5% complete for CAP11_APPC_37.15_R2.tagged_filter.fastq.gz


Approx 55% complete for CAP11_APPC_38.12_R1.tagged_filter.fastq.gz
Approx 90% complete for CAP10_APPC_35.11_R2.tagged_filter.fastq.gz
Approx 70% complete for CAP11_APPC_37.10_R1.tagged_filter.fastq.gz
Approx 85% complete for CAP10_APPC_36.06_R1.tagged_filter.fastq.gz
Approx 75% complete for CAP11_APPC_37.10_R2.tagged_filter.fastq.gz
Approx 35% complete for CAP11_APPC_38.12_R2.tagged_filter.fastq.gz
Approx 15% complete for CAP11_APPC_38.14_R2.tagged_filter.fastq.gz
Approx 25% complete for CAP11_APPC_37.15_R2.tagged_filter.fastq.gz
Approx 20% complete for CAP11_APPC_38.14_R1.tagged_filter.fastq.gz
Approx 40% complete for CAP10_APPC_35.14_R2.tagged_filter.fastq.gz
Approx 60% complete for CAP11_APPC_38.12_R1.tagged_filter.fastq.gz
Approx 95% complete for CAP10_APPC_35.11_R2.tagged_filter.fastq.gz
Approx 25% complete for CAP11_APPC_39.03_R1.tagged_filter.fastq.gz
Approx 75% complete for CAP11_APPC_37.10_R1.tagged_filter.fastq.gz
Approx 20% complete for CAP11_APPC_39.03_R2.tagged_filter.fast

Approx 60% complete for CAP11_APPC_37.15_R1.tagged_filter.fastq.gz
Approx 50% complete for CAP11_APPC_39.03_R2.tagged_filter.fastq.gz
Approx 80% complete for CAP11_APPC_39.03_R1.tagged_filter.fastq.gz
Approx 80% complete for CAP10_APPC_35.14_R1.tagged_filter.fastq.gz
Approx 90% complete for CAP11_APPC_38.12_R2.tagged_filter.fastq.gz
Approx 50% complete for CAP11_APPC_38.14_R2.tagged_filter.fastq.gz
Approx 65% complete for CAP11_APPC_38.14_R1.tagged_filter.fastq.gz
Approx 55% complete for CAP10_APPC_34.14_R1.tagged_filter.fastq.gz
Approx 55% complete for CAP10_APPC_34.14_R2.tagged_filter.fastq.gz
Approx 55% complete for CAP11_APPC_39.03_R2.tagged_filter.fastq.gz
Approx 80% complete for CAP10_APPC_35.14_R2.tagged_filter.fastq.gz
Approx 50% complete for CAP10_APPC_34.13_R2.tagged_filter.fastq.gz
Approx 65% complete for CAP11_APPC_37.15_R1.tagged_filter.fastq.gz
Approx 85% complete for CAP10_APPC_35.14_R1.tagged_filter.fastq.gz
Approx 50% complete for CAP10_APPC_34.13_R1.tagged_filter.fast

...

Approx 85% complete for CAP7_APPC_26.04_R1.tagged_filter.fastq.gz
Approx 30% complete for CAP9_APPC_31.21_R2.tagged_filter.fastq.gz
Approx 80% complete for CAP8_APPC_27.13_R2.tagged_filter.fastq.gz
Approx 50% complete for CAP8_APPC_28.28_R2.tagged_filter.fastq.gz
Approx 60% complete for CAP9_APPC_31.21_R1.tagged_filter.fastq.gz
Approx 70% complete for CAP8_APPC_27.07_R2.tagged_filter.fastq.gz
Approx 45% complete for CAP9_APPC_30.07_R2.tagged_filter.fastq.gz
Approx 20% complete for CAP8_APPC_29.29_R2.tagged_filter.fastq.gz
Approx 35% complete for CAP9_APPC_31.21_R2.tagged_filter.fastq.gz
Analysis complete for CAP8_APPC_27.08_R1.tagged_filter.fastq.gz
Approx 35% complete for CAP8_APPC_28.23_R2.tagged_filter.fastq.gz
Approx 65% complete for CAP9_APPC_31.21_R1.tagged_filter.fastq.gz
Approx 60% complete for CAP9_APPC_30.07_R1.tagged_filter.fastq.gz
Approx 95% complete for CAP7_APPC_24.14_R2.tagged_filter.fastq.gz
Approx 20% complete for CAP8_APPC_30.21_R1.tagged_filter.fastq.gz
Approx 40% c

Approx 55% complete for CAP8_APPC_27.12_R2.tagged_filter.fastq.gz
Approx 10% complete for CAP9_APPC_32.15_R2.tagged_filter.fastq.gz
Approx 85% complete for CAP9_APPC_30.07_R2.tagged_filter.fastq.gz
Analysis complete for CAP9_APPC_30.07_R1.tagged_filter.fastq.gz
Approx 30% complete for CAP8_APPC_30.21_R2.tagged_filter.fastq.gz
Started analysis of CAP9_APPC_33.02_R1.tagged_filter.fastq.gz
Approx 85% complete for CAP8_APPC_28.28_R1.tagged_filter.fastq.gz
Approx 20% complete for CAP9_APPC_32.15_R1.tagged_filter.fastq.gz
Approx 15% complete for CAP9_APPC_32.15_R2.tagged_filter.fastq.gz
Analysis complete for CAP8_APPC_27.07_R1.tagged_filter.fastq.gz
Approx 5% complete for CAP9_APPC_33.02_R1.tagged_filter.fastq.gz
Approx 80% complete for CAP8_APPC_28.28_R2.tagged_filter.fastq.gz
Approx 90% complete for CAP9_APPC_30.07_R2.tagged_filter.fastq.gz
Approx 95% complete for CAP8_APPC_27.07_R2.tagged_filter.fastq.gz
Started analysis of CAP9_APPC_33.02_R2.tagged_filter.fastq.gz
Approx 10% complete for

Started analysis of CAP9_APPC_33.08_R2.tagged_filter.fastq.gz
Approx 40% complete for CAP8_APPC_29.29_R1.tagged_filter.fastq.gz
Approx 90% complete for CAP9_APPC_33.08_R1.tagged_filter.fastq.gz
Approx 95% complete for CAP9_APPC_33.08_R1.tagged_filter.fastq.gz
Approx 5% complete for CAP9_APPC_33.08_R2.tagged_filter.fastq.gz
Analysis complete for CAP9_APPC_33.08_R1.tagged_filter.fastq.gz
Approx 10% complete for CAP9_APPC_33.08_R2.tagged_filter.fastq.gz
Approx 60% complete for CAP8_APPC_28.23_R1.tagged_filter.fastq.gz
Approx 15% complete for CAP9_APPC_33.08_R2.tagged_filter.fastq.gz
Approx 65% complete for CAP8_APPC_27.12_R2.tagged_filter.fastq.gz
Approx 20% complete for CAP9_APPC_33.08_R2.tagged_filter.fastq.gz
Approx 25% complete for CAP9_APPC_33.08_R2.tagged_filter.fastq.gz
Approx 40% complete for CAP9_APPC_33.02_R2.tagged_filter.fastq.gz
Approx 40% complete for CAP8_APPC_29.29_R2.tagged_filter.fastq.gz
Approx 30% complete for CAP9_APPC_33.08_R2.tagged_filter.fastq.gz
Approx 50% comple

Approx 95% complete for CAP9_APPC_32.15_R1.tagged_filter.fastq.gz
Approx 35% complete for CAP9_APPC_33.13_R1.tagged_filter.fastq.gz
Approx 90% complete for CAP8_APPC_27.12_R1.tagged_filter.fastq.gz
Approx 30% complete for CAP9_APPC_33.13_R2.tagged_filter.fastq.gz
Approx 95% complete for CAP9_APPC_32.15_R2.tagged_filter.fastq.gz
Approx 100% complete for CAP9_APPC_32.15_R1.tagged_filter.fastq.gz
Analysis complete for CAP9_APPC_32.15_R1.tagged_filter.fastq.gz
Approx 75% complete for CAP8_APPC_28.23_R1.tagged_filter.fastq.gz
Approx 15% complete for CAP9_APPC_32.24_R2.tagged_filter.fastq.gz
Approx 40% complete for CAP9_APPC_33.13_R1.tagged_filter.fastq.gz
Approx 30% complete for CAP9_APPC_31.10_R2.tagged_filter.fastq.gz
Approx 100% complete for CAP9_APPC_32.15_R2.tagged_filter.fastq.gz
Approx 35% complete for CAP9_APPC_33.13_R2.tagged_filter.fastq.gz
Analysis complete for CAP9_APPC_32.15_R2.tagged_filter.fastq.gz
Approx 55% complete for CAP8_APPC_30.21_R2.tagged_filter.fastq.gz
Approx 45% c

Approx 35% complete for CAP9_APPC_32.24_R2.tagged_filter.fastq.gz
Approx 85% complete for CAP8_APPC_29.29_R1.tagged_filter.fastq.gz
Approx 50% complete for CAP10_APPC_34.14_R2.tagged_filter.fastq.gz
Approx 85% complete for CAP8_APPC_29.29_R2.tagged_filter.fastq.gz
Approx 90% complete for CAP8_APPC_30.21_R1.tagged_filter.fastq.gz
Approx 85% complete for CAP9_APPC_32.03_R1.tagged_filter.fastq.gz
Approx 90% complete for CAP8_APPC_30.21_R2.tagged_filter.fastq.gz
Approx 75% complete for CAP9_APPC_32.03_R2.tagged_filter.fastq.gz
Approx 55% complete for CAP10_APPC_34.14_R2.tagged_filter.fastq.gz
Approx 90% complete for CAP8_APPC_29.29_R1.tagged_filter.fastq.gz
Approx 45% complete for CAP9_APPC_32.24_R1.tagged_filter.fastq.gz


In [ ]:
#visualize sequencing quality for all samples in trimmed_reads directory. check that all barcode sequences are removed
!multiqc .

In [ ]:
#make bwa index for reference fasta to use in mapping
!mkdir mapping
%cd ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/mapping
!bwa index /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/Genome/reference.fasta

In [ ]:
#map using default parameters but add -M for picard compatibility, make sure to use the tagged/filtered
!for i in $(cat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/filenames); do bwa mem -t 60 -M -R "@RG\tID:$i\tSM:$i\tPL:Illumina" /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/fastq_screen/mapping/reference.fasta $i.R1.fastq.gz $i.R2.fastq.gz  2> bwa.$i.log | samtools sort -@$NUMProc -o $i-RG.bam 2>$i.bam.log; done  

In [ ]:
#make sure to check before running this, can't remember where .sai files are made...
#join pairs, convert from sam to bam files in screen -S samsort
#screen -S samsort
for i in $(cat APPC_sequencing/ultraplex_out/fastq_screen/filenames);
do bwa sampe /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/mapping/reference.fasta $i\.read1.sai $i\.read2.sai $i\.R1.tagged_filter.fastq.gz $i\.R2.tagged_filter.fastq.gz | samtools view -bS - >  ~/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/$i\.bam; done



In [ ]:
#remove duplicate reads with picard
java -Xms4g -jar /local/home/katrinac/miniconda3/envs/commandline/bin/picard.jar MarkDuplicates I={}-R2.tagged_filter.fastq.gz O={}-RGmd.bam M={}_dup_metrics.txt OPTICAL_DUPLICATE_PIXEL_DISTANCE=2500 TAGGING_POLICY=OpticalOnly &> md.{}.log   

In [ ]:
#January 24, 2022 using fastp to trim adapters because trimmomatic left the barcode in read 2 file
for i in $(cat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/filenames); 
    do fastp -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/$i.R1.fq.gz -I /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/$i.R2.fq.gz -o /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/fastp/$i.R1.fastp.fq.gz -O /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/fastp/$i.R2.fastp.fq.gz; 
    done



In [ ]:
#January 25, there's still barcodes in the beginning of the read 2 files after fastp adapter trimming. I'm going to try ultraplex instead.
cp ~/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_03_26_SEQ1/bcsplit/*.gz ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/
cp ~/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_04_16_SEQ2/bcsplit/*.gz ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/

for i in $(cat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/cap_filenames); 
    do ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/$i-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/$i-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/APPCBarcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/ultraplex_out; 
    done


In [ ]:
#this worked! do this for all capture pools, rename, move this line to the demultiplex notebook, and then redo the fastq, mapping, picard, then angsd and genotype calling
ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP1-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP1-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/Cap1_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/ultraplex_out 